In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CorpusforDDL_compute


In [2]:
import pandas as pd
import re
import unicodedata
import pickle
from collections import Counter

In [3]:
# 讀入 ASBC txt 檔
asbc_path = 'CorpusforDDL/ASBC_去XML標記'
all_files = os.listdir(asbc_path)

asbc_corpus = []
for f in all_files:
  with open(f'{asbc_path}/{f}') as f:
      lines = f.readlines()
      asbc_corpus.append(lines)

In [4]:
# 前處理
def preprocess_asbc(string):

  clean_string = unicodedata.normalize('NFKC', string) # 全形轉半形
  clean_string = re.sub(r'\n', '', clean_string) # 移除換行符號
  clean_string = re.sub(r'\[\+[A-z]+\]', '', clean_string) # 移除特徵標記

  clean_string = clean_string.translate(str.maketrans({',': '，', 
                                                       '!': '！', 
                                                       '?': '？',
                                                       ':': '：',
                                                       ';': '；'})) # 部分符號改回全形
  return clean_string

In [5]:
asbc_corpus_preprocessed = []

for text in asbc_corpus:
  preprocessed = [preprocess_asbc(string) for string in text]
  joined = ''.join(preprocessed)
  splitted = re.split(r'(?<=[。！？])', joined)

  asbc_corpus_preprocessed.append(splitted)

In [6]:
asbc_test = asbc_corpus_preprocessed[0] # 先取一個檔案當例子
asbc_test[0]

'碰到(VC) 這(Nep) 種(Nf) 情形(Na) ，(COMMACATEGORY)真(D) 是(SHI) 會(D) 令(VL) 人(Na) 又(Caa) 氣憤(VH) 又(Caa) 苦惱(VH) 。'

In [7]:
def asbc_get_sentence(sentence):

  sentence_words = re.sub(r'\([A-z0-9]+\)', '', sentence)
  sentence_words = re.sub(r'[^\w\s]', '', sentence_words).strip().split(' ')
  sentence_words = list(filter(None, sentence_words))

  return sentence_words

In [8]:
asbc_test_sent = [asbc_get_sentence(sentence) for sentence in asbc_test]
asbc_test_sent[0]

['碰到', '這', '種', '情形', '真', '是', '會', '令', '人', '又', '氣憤', '又', '苦惱']

In [9]:
asbc_test_text = []

for sent in asbc_test:
  res = re.sub(r'\([A-z0-9_]+\)', '', sent)
  res = res.replace(' ', '')
  asbc_test_text.append(res)

In [10]:
asbc_test_text[:5]

['碰到這種情形，真是會令人又氣憤又苦惱。',
 '相信大多數的母親也都有過類似的困擾，因為我們的生活環境逐漸被色情電影、錄影帶、畫片、書刊所污染。',
 '尤其在「笑貧不笑娼」的社會風氣裡，有些唯利是圖的不法商人，更是以誇張的色情表演和刺激的畫面來招攬生意。',
 '要我們這些做父母的完全禁止孩子們去接觸，可真是防不勝防。',
 '縱使你的孩子現在沒看過，以後他同樣會有機會接觸，所以，我們要做的並不是彌補的工作，而是如何「正視」地處理。']

## 句子長度
計算方法：算出每一個句子中含有幾個詞彙

In [11]:
sentence_length = [len(sentence) for sentence in asbc_test_sent]
sentence_length[:10]

[13, 25, 31, 16, 31, 33, 3, 40, 24, 7]

## 詞頻
計算方法：參考 COCT 詞頻資料，以詞頻 1200 為標準，回傳每一個句子中含有幾個高頻詞、幾個低頻詞，還有句子中每一個詞的詞頻

In [12]:
# 讀入 COCT txt 檔
coct_path = 'data/coct_frequency_list_2019.txt'
with open(coct_path) as f:
    lines = f.readlines()
    lines = [line.rstrip('\n') for line in lines]

In [13]:
coct_data = [x.split('\t') for x in lines[5:len(lines)-2]]
coct_data[:5]

[['1', '，', '23348406'],
 ['2', '的', '16029838'],
 ['3', '。', '11412297'],
 ['4', '是', '4642407'],
 ['5', '一', '3701681']]

In [14]:
keys = [x[1] for x in coct_data]
vals = [int(x[2]) for x in coct_data]
print(keys[:5])
print(vals[:5])

['，', '的', '。', '是', '一']
[23348406, 16029838, 11412297, 4642407, 3701681]


In [15]:
coct_data_dict = dict(list(zip(keys, vals)))
coct_data_dict.get('我')

3033032

In [16]:
def get_high_low_freq(sentence):

  freq = []
  for word in sentence:
    try:
      f = coct_data_dict[word]
    except:
      f = 0 # 若是在 sorted_wordfreq_dict 中找不到該詞的頻率資料，則將其頻率指定為 0
    freq.append(f)

  high_low = ['High' if f > 100 else 'Low' for f in freq] # 若在 ASBC 中詞頻大於 100，則視為高頻詞 (High)

  return dict(Counter(high_low))

In [17]:
get_high_low_freq(asbc_test_sent[5])

{'High': 32, 'Low': 1}

In [18]:
high_low_freq = [get_high_low_freq(sentence) for sentence in asbc_test_sent]
high_low_freq[:10]

[{'High': 13},
 {'High': 25},
 {'High': 31},
 {'High': 16},
 {'High': 31},
 {'High': 32, 'Low': 1},
 {'High': 3},
 {'High': 40},
 {'High': 24},
 {'High': 7}]

In [19]:
def get_word_freq(sentence):

  word_freq = []

  for word in sentence:
    try:
      freq = coct_data_dict[word]
    except:
      freq = 'Unknown'
    res = f'{word}({freq})'
    word_freq.append(res)

  word_freq = ' '.join(word_freq)

  return word_freq

In [20]:
word_freq = [get_word_freq(sentence) for sentence in asbc_test_sent]
word_freq[0]

'碰到(15360) 這(2114032) 種(701563) 情形(35199) 真(137497) 是(4642407) 會(1014977) 令(128742) 人(1550512) 又(535555) 氣憤(2937) 又(535555) 苦惱(3892)'

## 詞彙等級
計算方法：參考國教院詞語分級表，回傳每一個句子中，各級別的詞彙分別有幾個

In [21]:
# 參考國教院詞語分級表
naer_word_list = pd.read_excel('data/臺灣華語文能力基準詞語表_111-09-20.xlsx')
naer_word_list

,序號,詞語,等別,級別,情境,書面字頻(每百萬字),口語字頻(每百萬字),簡編本系統號,參考注音,參考漢語拼音
0,1,愛,基礎,第1級,核心詞,535,681,"[['愛', ['39542']]]",ㄞˋ,ài
1,2,吧,基礎,第1級,核心詞,706,748,"[['吧', ['32', '103']]]",˙ㄅㄚ,ba
2,3,八,基礎,第1級,核心詞,214,163,"[['八', ['1']]]",ㄅㄚ,bā
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà
4,5,百,基礎,第1級,核心詞,108,77,"[['百', ['157', '334']]]",ㄅㄞˇ,bǎi
...,...,...,...,...,...,...,...,...,...,...
14462,14463,左右手,精熟,第7級,NaN,3,2,"[['左右手', ['37450']]]",ㄗㄨㄛˇ ㄧㄡˋ ㄕㄡˇ,zuǒ yòu shǒu
14463,14464,坐鎮,精熟,第7級,NaN,5,2,"[['坐鎮', ['37472']]]",ㄗㄨㄛˋ ㄓㄣˋ,zuò zhèn
14464,14465,佐證,精熟,第7級,NaN,5,4,"[['佐證', ['37456']]]",ㄗㄨㄛˇ ㄓㄥˋ,zuǒ zhèng
14465,14466,坐姿,精熟,第7級,NaN,5,1,[],ㄗㄨㄛˋ ㄗ,zuò zī


In [22]:
naer_word_list = naer_word_list.assign(word_1=naer_word_list['詞語'].str.split('/')).explode('詞語')
naer_word_list = naer_word_list.explode('word_1') # 把用 / 連接的詞語切開做成新的一列
naer_word_list

,序號,詞語,等別,級別,情境,書面字頻(每百萬字),口語字頻(每百萬字),簡編本系統號,參考注音,參考漢語拼音,word_1
0,1,愛,基礎,第1級,核心詞,535,681,"[['愛', ['39542']]]",ㄞˋ,ài,愛
1,2,吧,基礎,第1級,核心詞,706,748,"[['吧', ['32', '103']]]",˙ㄅㄚ,ba,吧
2,3,八,基礎,第1級,核心詞,214,163,"[['八', ['1']]]",ㄅㄚ,bā,八
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà,爸爸
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà,爸
...,...,...,...,...,...,...,...,...,...,...,...
14462,14463,左右手,精熟,第7級,NaN,3,2,"[['左右手', ['37450']]]",ㄗㄨㄛˇ ㄧㄡˋ ㄕㄡˇ,zuǒ yòu shǒu,左右手
14463,14464,坐鎮,精熟,第7級,NaN,5,2,"[['坐鎮', ['37472']]]",ㄗㄨㄛˋ ㄓㄣˋ,zuò zhèn,坐鎮
14464,14465,佐證,精熟,第7級,NaN,5,4,"[['佐證', ['37456']]]",ㄗㄨㄛˇ ㄓㄥˋ,zuǒ zhèng,佐證
14465,14466,坐姿,精熟,第7級,NaN,5,1,[],ㄗㄨㄛˋ ㄗ,zuò zī,坐姿


In [23]:
levels = naer_word_list['級別'].values
word_level_dict = pd.Series(levels, index=naer_word_list['word_1']).to_dict()

In [24]:
def get_word_level(sentence):

  levels = []
  for word in sentence:
    try:
      level = word_level_dict[word]
    except:
      level = 'Unknown' # 如果找不到該詞彙，級別設為 Unknown
    levels.append(level)
  
  return dict(Counter(levels))

In [25]:
word_level = [get_word_level(sentence) for sentence in asbc_test_sent]
word_level[0]

{'第4級': 1, '第1級': 3, 'Unknown': 1, '第3*級': 1, '第1*級': 4, '第5級': 2, '第6級': 1}

## 詞彙長度
計算方法：算出每一個句子中有幾個「長度大於三個字」的詞彙

In [26]:
def get_long_word_count(sentence):

  long_word_count = 0
  for word in sentence:
    if len(word) >= 3:
      long_word_count += 1
  
  return long_word_count

In [27]:
long_word_count = [get_long_word_count(sentence) for sentence in asbc_test_sent] # 算出該句子有幾個「長度大於三個字」的詞彙
long_word_count[:10]

[0, 2, 1, 1, 0, 1, 1, 1, 0, 0]

## 完整的句子
計算方法：
1. 以。？！結尾
1. 至少含有一個動詞

In [28]:
# verb_list = ['VA', 'VAC', 'VB', 'VC', 'VCL', 'VD', 'VE', 
#              'VF', 'VG', 'VH', 'VHC', 'VI', 'VJ', 'VK', 
#              'VL', 'V_2']
# verb_list = [f'({x})' for x in verb_list]
# verb_list

In [29]:
def get_complete_sentence(sentence):

  if re.search(r'。|？|！', sentence) and 'V' in sentence:
    res = 'Y'
  else:
    res = 'N'

  return res

In [30]:
is_complete_sentence = [get_complete_sentence(sentence) for sentence in asbc_test]
is_complete_sentence[:10]

['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y']

## 完整的語境
計算方法：
1. 用 `Caa` `Cab` `Cba` `Cbb` 判定

In [31]:
def get_complete_context(sentence):

  if re.search(r'Caa|Cab|Cba|Cbb', sentence):
    res = 'Y'
  else:
    res = 'N'

  return res

In [32]:
is_complete_context = [get_complete_context(x) for x in asbc_test]
is_complete_context[:10]

['Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'N']

## Black List
計算方法：判斷是否包含
1. 連續八個以上數字或數詞
1. 十個以上包含 `.` 的非中文字串 (網址)

In [33]:
def get_blacklist(sentence):

  chinese_num_upper = r'[零壹貳參肆伍陸柒捌玖拾佰仟]{8}'
  chinese_num_lower = r'[○一二三四五六七八九十廿百千]{8}' 
  url = r'(www|http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

  check_chinese_num_upper = re.search(chinese_num_upper, sentence)
  check_chinese_num_lower = re.search(chinese_num_lower, sentence)
  check_digit = re.search(r'[0-9]{8}', sentence)
  check_url = re.search(url, sentence)

  if check_chinese_num_upper or check_chinese_num_lower or check_digit or check_url:
    res = 'Y'
  else:
    res = 'N'

  return res

In [34]:
is_blacklist = [get_blacklist(x) for x in asbc_test_text]
is_blacklist[:10]

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

## Grey List

計算方法：含有 `Nb` 的句子

In [35]:
def get_greylist(sentence):

  if re.search(r'Nb', sentence):
    res = 'Y'
  else:
    res = 'N'

  return res

In [36]:
is_greylist = [get_greylist(x) for x in asbc_test_text]
is_greylist[:10] 

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']

In [37]:
asbc_df = pd.DataFrame({'sentence': asbc_test_text,
                        'sentence_preprocessed': [' '.join(sent) for sent in asbc_test_sent],
                        'sentence_length': sentence_length,
                        'word_freq': word_freq,
                        'high_low_freq': high_low_freq,
                        'word_level': word_level,
                        'long_word_count': long_word_count,
                        'is_complete_sentence': is_complete_sentence,
                        'is_complete_context': is_complete_context,
                        'is_blacklist': is_blacklist,
                        'is_greylist': is_greylist})

In [38]:
asbc_df.to_csv('results/asbc/asbc_parameters_example.csv', index = False)

## Concordance

In [39]:
from nltk.text import Text

In [40]:
target_words = ['難得', '畢竟', '的確', '難免', '總是', '有助於']
corpus = [item for sublist in asbc_test_sent for item in sublist]
text = Text(corpus)
dfs = []

for word in target_words:

  con_list = text.concordance_list(word)
  right_word = [x.right[0] for x in con_list]
  left_word = [x.left[-1] for x in con_list]
  context = [x.left + [word] + x.right for x in con_list]
  context = [' '.join(x) for x in context]

  df = pd.DataFrame({'left_word': left_word,
                     'target_word': word,
                     'right_word': right_word,
                     'context': context})
  dfs.append(df)

In [41]:
concordance_df = pd.concat(dfs)
concordance_df.to_csv('results/asbc/asbc_concordance_df.csv', index = False)